# ML в Биологии
## Аугментация


In [ ]:
import os
import gc
import time
import glob
import requests
from tqdm.notebook import tqdm
from collections import defaultdict

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms

from IPython.display import clear_output
%matplotlib inline

sns.set(font_scale=1.2, style='whitegrid')

# device_num = 0
# torch.cuda.set_device(device_num)

In [ ]:
device = f"cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(device)

cuda


## 1. Аугментация <a id='augmentations'> </a>

![](https://sun9-5.userapi.com/c857216/v857216818/1815e6/OxQ8e3-o0oU.jpg)

**Дополнение данных / Аугментация данных / Data augmentations** &mdash; это метод, направленный на увеличение размеров обучающей выборки. Дополнение обучающей выборки разнообразными, "хорошими" и "плохими" примерами, позволяет получить модель более устойчивую на тестовых данных, так как для неё в тестовых данных будет меньше "неожиданностей".

Благодаря модулю `torchvision.transforms` агументации можно делать очень просто. Про все реализованные в библиотеке преобразования можно почитать [здесь](https://pytorch.org/vision/stable/transforms.html#transforms-scriptability). Мы рассмотрим наиболее распространенные классы аугментаций.

* `RandomAffine(degrees, translate=None, scale=None, shear=None, interpolation=<InterpolationMode.NEAREST: 'nearest'>, fill=0, fillcolor=None, resample=None)` &mdash; **случайное афинное преобразование** с сохранением центра. `degrees` &mdash; градус вращения. `translate` &mdash; смещение. `scale` &mdash; масштаб. Подробнее в документации.


* `ColorJitter(brightness=0, contrast=0, saturation=0, hue=0)` &mdash; *случайное* изменение **яркости** / brightness, **контраста** / contrast, **насыщенности** / saturation и **тонов** / hue цветов. Если на вход приходит torch.Tensor, то его размерность дожна быть `[..., 3, H, W]`. Если PIL.Image, то без альфа-канала. Каждый из параметров может быть быть задан в виде float числа: param, или пары float чисел: min, max. Значение парметра выбирается случайно из отрезка `[1 - param, 1 + param]` или `[min, max]` для brightness, contrast, saturation. Значение парметра должно быть нотрицательным. Значение парметра hue выбирается случайно из отрезка `[-hue, hue]` или `[min, max]`. При этом значение 0<= `hue <= 0.5` or `-0.5 <= min <= max <= 0.5`.  


* `CenterCrop(size)` &mdash; вырезает **прямоугольную область** размером `size[0] x size[1]`, если size задан туплом, если `size` задан числом &mdash; `size x size` **из центра картинки**.


* `GaussianBlur(kernel_size, sigma)` &mdash; *случайное* **гауссовское размытие изображения**. `kernel_size` &mdash; размер гауссовского ядра. `sigma` &mdash; стандартное отклонение. `sigma` может быть задано в виде чила, тогда парметр фиксирован, или в виде тупла in, max, тогда оно выбирается случайно из отрезка `[min, max]`.  


* `Grayscale(num_output_channels=1)` и `RandomGrayscale`(p=0.1) &mdash; неслучайная и *случайная* **трансформации картинки в ч/б формат**. Grayscale имеет парметр `num_output_channels`, который означет количество каналов на выходе, он может быть равен 1 или 3. RandomGrayscale имеет парметр p, который равен вероятности применения преобразования. Тензор на выхоже будет иметь столько же каналов, сколько тензор на входе.   


* `Normalize(mean, std, inplace=False)` &mdash; **нормализация тензора картинки** с заданными средним и отклонением для каждого канала. То есть `mean = (mean[1], ..., mean[n])`, `std = (std[1], ..., std[n])`, где `n` &mdash; количество каналов. Не поддерживает PIL.Image формат!   


* `RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=<InterpolationMode.BILINEAR: 'bilinear'>)` &mdash; **случайное обрезание картинки** со случайным выбором размера и соотношения сторон и последующим **увеличеним картинки до первонального размера**.  

* `Resize(size, interpolation=<InterpolationMode.BILINEAR: 'bilinear'>)`  &mdash; **изменение размеров кратинки**. Если `size` задан числом, то наименьшая из размерностей картинки приобретает размер `size`. Иначе, если размер задан парой, то размер картинки становится равным `size[0] x size[1]`.

Для того, чтобы получить преобразование, которого нет в модуле torchvision.transforms можно использовать `Lambda` преобразование. Например, получить гауссовский шум на изображении можно так:
```
Lambda(lambda x : x + torch.randn_like(x))
```

Выше перечисленные трансформации применяются к данным типа PIL.Image или torch.Tensor,на выходе выдают соотвествующий формат. Для того, чтобы в через тарнсформации получить PIL.Image, можно использовать класс `ToPILImage`, для того, чтобы получить torch.Tensor &mdash; `ToTensor`. Эти классы в методе forward могут использовать torch.Tensor, np.ndarray и PIL.Image, np.ndarray соотвественно.

Чтобы объединить несколько трансформаций можно использовать `Compose`(transforms), где transforms &mdash; список из объектов коассов преобразований.

### Практика по аугментации

In [ ]:
from PIL import Image

In [ ]:
# скачиваем изображения
! wget https://upload.wikimedia.org/wikipedia/ru/thumb/2/24/Lenna.png/1920px-Lenna.png
! wget https://miptstats.github.io/images/logo_bottom.png

--2024-09-21 16:00:06--  https://upload.wikimedia.org/wikipedia/ru/thumb/2/24/Lenna.png/1920px-Lenna.png
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5545662 (5.3M) [image/png]
Saving to: ‘1920px-Lenna.png.1’

1920px-Lenna.png.1  100%[===================>]   5.29M  29.5MB/s    in 0.2s    

2024-09-21 16:00:07 (29.5 MB/s) - ‘1920px-Lenna.png.1’ saved [5545662/5545662]

--2024-09-21 16:00:07--  https://miptstats.github.io/images/logo_bottom.png
Resolving miptstats.github.io (miptstats.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to miptstats.github.io (miptstats.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212943 (208K) [image/png]
Saving to: ‘logo_bottom.png.1’

logo_bottom.png.1   100%[=============

Выберите 3 преобразования, которые вам понравятся [отсюда](https://pytorch.org/vision/stable/transforms.html#transforms-on-pil-image-and-torch-tensor) 3 преобразования, прмените его к исходному изображению и объясните, что это преобразование делает.

Пример применения:

In [1]:
# transform = transforms.RandomAffine(degrees=75)
# transform(image)

In [2]:
#Поворачивает картинку
# transform = transforms.RandomRotation(degrees = 50)
# transform(image)

In [3]:
# Искажает изображение "в перспективе"
# transform = transforms.RandomPerspective()
# transform(image)

**Выводы:**

1. `transforms.RandomAffine(degrees=75)`

   Поворачивает изображение на случайный угол (до 75 градусов) и может масштабировать или смещать его. Это помогает модели научиться распознавать объекты, независимо от их ориентации.

2. `transforms.RandomRotation(degrees=50)`

   Поворачивает изображение на случайный угол в диапазоне от -50 до +50 градусов. Это добавляет вариативность данных и делает модель более устойчивой к разным ракурсам объекта.

3. `transforms.RandomPerspective()`

   Искажает изображение с применением перспективной трансформации, создавая эффект изменения точки обзора. Это позволяет модели быть более адаптивной к изображениям, снятым под разными углами.

Все эти трансформации увеличивают разнообразие обучающей выборки, что помогает улучшить обобщающие способности модели.